In [1]:
import sys
import os
import torch
from optimizer import Optimizer
from config import Config
from IPython.display import Image, display
from IPython.display import Image
import os, glob

c:\Users\dani_\anaconda3\envs\faceNext\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [2]:
sys.path.insert(0,'/content/NextFace') #verify ur path

config = Config()
config.fillFromDicFile('./optimConfig.ini')
# config.device = 'cuda' # torch not compiled with cuda
config.path = './baselMorphableModel/' #verify ur path


loading optim config from:  ./optimConfig.ini


In [3]:


# imagePath = './input/detailled_faces_unsplash/BikerMan.jpg'
imagePath = './output/Bikerman.jpg/master_checkpoints/mitsuba_ref.png'

outputDir = './output/' + os.path.basename(imagePath.strip('/'))
torch.cuda.set_device(0)
optimizer = Optimizer(outputDir ,config)

Loading Basel Face Model 2017 from ./baselMorphableModel//morphableModel-2017.pickle...
loading mesh normals...
loading uv parametrization...
loading landmarks association file...
creating sampler...
loading image from path:  C:/Users/dani_/Desktop/repos/NextFace/output/Bikerman.jpg/diffuseMap_0.png


c:\Users\dani_\anaconda3\envs\faceNext\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
#run the optimization now 
# optimizer.run(imagePath,doStep1=True,doStep2=True, doStep3=False)
optimizer.run(imagePath,checkpoint='C:/Users/dani_/Desktop/repos/NextFace/output/Bikerman.jpg/master_checkpoints/stage1_output.pickle', doStep1=False,doStep2=True, doStep3=False)

loading image from path:  ./output/Bikerman.jpg/master_checkpoints/mitsuba_ref.png
detecting landmarks using: fan


c:\Users\dani_\Desktop\repos\NextFace\landmarksfan.py:38: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.tensor(landmarks, device = self.device)
init camera pose...
resuming optimization from checkpoint:  C:/Users/dani_/Desktop/repos/NextFace/output/Bikerman.jpg/master_checkpoints/stage1_output.pickle
2/3 => Optimizing shape, statistical albedos, expression, head pose and scene light...
  0%|          | 0/401 [00:00<?, ?it/s]c:\Users\dani_\Desktop\repos\NextFace\optimizer.py:426: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tenso

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\matrix_operations.cpp:67: error: (-215:Assertion failed) src[i].dims <= 2 && src[i].rows == src[0].rows && src[i].type() == src[0].type() in function 'cv::hconcat'


Results

In [ ]:
#display results
# Directory containing the images
dir_path = outputDir + '/debug/Baseline/vertex_based/'

# Get a list of all the .png images in the directory
image_files = glob.glob(os.path.join(dir_path, "*.png"))

# Display each image in turn
for image_file in image_files:
    print(f"Displaying image: {os.path.basename(image_file)}")
    display(Image(filename=image_file))


In [ ]:
# display meshes in polyscope
import polyscope as ps
import trimesh
import glob
import math

display_meshes = ["vertex_based_step2_iter0.obj", "vertex_based_step2_iter100.obj","vertex_based_step2_iter200.obj","vertex_based_step2_iter300.obj","vertex_based_step2_iter400.obj"]  # replace this with your mesh names


# Get a list of all .obj files in the directory
obj_files = glob.glob(os.path.join(outputDir + '/debug/mesh/', "*.obj"))

# Filter obj_files to only include the ones in display_meshes
obj_files = [file for file in obj_files if os.path.basename(file) in display_meshes]

# Define a folder where the screenshots will be saved
screenshot_folder = outputDir+ "/debug/screenshot_folder"  # replace with your desired folder

# Check if the folder exists, if not create it
os.makedirs(screenshot_folder, exist_ok=True)

# Initialize polyscope
ps.init()
# enable auto centering and scaling
# ps.set_autocenter_structures(True)
# ps.set_autoscale_structures(True)
ps.set_up_dir("neg_y_up")
# ps.set_bounding_box(-1.0,2.0)
# ps.set_SSAA_factor(4)
# ps.set_automatically_compute_scene_extents(True)
# ps.set_autoscale_structures(True)
# ps.set_autocenter_structures(True)
# ps.set_ground_plane_height_factor(1.0,True)
ps.set_ground_plane_mode("none")

# # Show the mesh
# ps.show()
# Load and register each mesh to polyscope
for idx, obj_file in enumerate(obj_files):
    # mesh_name = os.path.basename(obj_file)  # get the name of the mesh
    mesh_name = os.path.basename(obj_file)  # get the name of the mesh

    # Load the mesh from an obj file using trimesh
    mesh = trimesh.load_mesh(obj_file)
    # Register the mesh to polyscope
    ps_mesh = ps.register_surface_mesh(mesh_name, mesh.vertices, mesh.faces)
    # Reset the transformation on the mesh
    ps_mesh.reset_transform()
    # Set a color for the mesh
    color = torch.tensor([0.5, 0.5, 0.5]).numpy()  # set a fixed color
    ps_mesh.set_color(color)
    # Reset the view
    ps.reset_camera_to_home_view()
    # Show the mesh
    ps.show()

    # Take a screenshot and save it to the specified folder
    screenshot_name = os.path.join(screenshot_folder, f"{mesh_name}_screenshot.png")
    ps.screenshot(screenshot_name)

    # Remove the mesh from polyscope to prepare for the next one
    ps.remove_all_structures()

In [ ]:
#display results
# Directory containing the images
dir_path = outputDir + '/debug/screenshot_folder/'

# Get a list of all the .png images in the directory
image_files = glob.glob(os.path.join(dir_path, "*.png"))

# Display each image in turn
for image_file in image_files:
    print(f"Displaying image: {os.path.basename(image_file)}")
    display(Image(filename=image_file))
